In [129]:
from rdflib import *
import json
import re
ns = "https://buffalo.edu/lfocourse/assignment1p2/"


def get_resource(local):
    label = re.sub("\W+","",local)    
    return URIRef(f'{ns}{label}') , Literal(local, lang='en')

constraint_set = {}
for p in dir(OWL):
    if p.__str__().find("operty") >=0 : 
        f=re.findall("#(.+)Property",p.__str__())
        if len(f) > 0:
            constraint_set[f[0]] = p
##print(json.dumps(constraint_set, indent=2))

def get_prop(selector):
    ptype = constraint_set.get(selector)
    res, lab = get_resource(selector.lower())
    return res,lab, ptype

def subproperty(constraint_1, constraint_2):
    text = f"Suppose B owl:subPropertyOf A, A is {constraint_1} and B is {constraint_2}."
    return f"<p>{text}</p>"

def inverseof(constraint_1, constraint_2):
    text =  f"Suppose B owl:inverseOf A, A is {constraint_1} and B is {constraint_2}."
    return f"<p>{text}</p>"
def owl_fn(constraint_1, constraint_2,ipl):
    return f"{constraint_1}_{ipl}_{constraint_2}.ttl"

def get_table(constraint_1, constraint_2): 
    df = "OK"
    grid = []
    grid.append(["",constraint_2,"","",""])
    grid.append([constraint_1,"A","Ai","B","Bi"])
    grid.append(["A",df,df,df,df])
    grid.append(["Ai",df,df,df,df])
    grid.append(["B",df,df,df,df])
    grid.append(["Bi",df,df,df,df]) 
    ogrid= []
    for g in grid:
        ogrid.append("<tr>" + "".join([f"<td>{x}</td>" for x in g]) + "</tr>") 
    #return ogrid    
    return "<table>\n" + "\n".join(ogrid) + "\n</table>"


def get_model(constraint_1, constraint_2, inter_prop):
    
    model = Graph()
    model.bind("dev",ns)

    Ao, Al , At = get_prop(constraint_1) 
    Bo, Bl , Bt = get_prop(constraint_2) 
    model.add((Ao,RDF.type, At))
    model.add((Ao,RDFS.label,Al))    
    model.add((Bo,RDF.type, Bt))
    model.add((Bo,RDFS.label,Bl)) 
    model.add((Bo,inter_prop,Ao))
    
    o, ol = get_resource("Test Thing")
    model.add((o,RDF.type,OWL.Class))
    model.add((o,RDFS.label,ol))
    
    x, xl = get_resource('x') 
    model.add((x,RDF.type,o))
    model.add((x,RDFS.label, xl))
    y, yl = get_resource('y') 
    model.add((y,RDF.type,o))
    model.add((y,RDFS.label, yl))
    z, zl = get_resource('z')     
    model.add((z,RDF.type,o))
    model.add((z,RDFS.label, zl))
    return model

In [130]:
F="Functional"
IF = "InverseFunctional"
T = "Transitive"
S = "Symmetric"
AS = "Asymmetric"
R = "Reflexive"
IR = "Irreflexive"
combinations = """
F,IF,S,AS,R,IR
IF,S,AS,R,IR
T,S,R
S,R,IR
AS,IR"""
C= [ c.split(",") for c in combinations.split("\n") if len(c)>0]

readme = []
if True:
    for c in C:
        for i in range(1,len(c)):
            c1 = eval(c[0])
            c2 = eval(c[i])
            title = f"## {c1} and {c2}"
            table = get_table(c1,c2)
            
            
            inv_model_a_b = get_model(c1,c2,OWL.inverseOf)
            inv_stmt_a_b = inverseof(c1,c2)
            inv_owl_a_b = owl_fn(c1,c2,"inverseOf")
            
            subprop_model_a_b = get_model(c1,c2,RDFS.subPropertyOf)
            subprop_stmt_a_b = subproperty(c1,c2)
            subprop_owl_a_b = owl_fn(c1,c2,"subPropertyOf")
            ## do we need to study this in both directions? 
            ##subprop_model_b_a = get_model(c1,c2,RDFS.subPropertyOf)            
            
            readme.append(title)
            readme.append("\n")
            readme.append(table)
            readme.append("\n")
            readme.append(subprop_stmt_a_b) 
            readme.append(f'<a href="{subprop_owl_a_b}">{subprop_owl_a_b}</a>')            
            readme.append("\n")
            readme.append(inv_stmt_a_b) 
            readme.append(f'<a href="{inv_owl_a_b}">{inv_owl_a_b}</a>')            
            readme.append("\n")
            readme.append("\n")  
            open(subprop_owl_a_b,'w').write(subprop_model_a_b.serialize(format = 'ttl'))            
            open(inv_owl_a_b,'w').write(inv_model_a_b.serialize(format = 'ttl'))
            
##print(get_model("Asymmetric","Reflexive",OWL.inverseOf).serialize(format='ttl') ) 
        
open("project1_part2.md",'w').write("".join(readme)) 


10950